In [ ]:
#!wget --continue http://www.lrc.ic.unicamp.br/~ferraroni/datasets/adrenaline/conversations.zip
#!unzip conversations.zip >> log.out

In [ ]:
!pip install --quiet "scikit-learn" "datasets" "scipy" "torchmetrics>=0.3" "transformers==4.9.0" "torch>=1.9" "pytorch-lightning>=1.3" "huggingface-hub==0.0.12"

In [1]:
import os
import tqdm
import glob
import pandas as pd

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, load_metric

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained("pierreguillou/gpt2-small-portuguese")
model = GPT2LMHeadModel.from_pretrained("pierreguillou/gpt2-small-portuguese")

In [5]:
from datasets import ClassLabel
import random
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
files = glob.glob("clear_threads/*_*.tsv")

frames = []
counter = 1

for file in tqdm.notebook.tqdm(files):
    counter += 1
    df = pd.read_csv(file, delimiter='\t', quoting=3, header=None, names=["timestamp", "id", "text"])
    df.drop(columns=['timestamp'], inplace=True)
    df['reply'] = df['text'].shift(-1)
    df = df.iloc[:-1]
    os.remove(file)
    frames.append(df)
    if counter % 1000 == 0:
        out = pd.concat(frames, axis=0, ignore_index=True)
        out.dropna(axis=0, inplace=True)
        frames = []
        out.to_parquet(file.replace('.tsv', '.parquet'))
        del out

In [7]:
train_files, validation_files = train_test_split(glob.glob("clear_threads/11*.parquet"), test_size=0.1)

In [8]:
datasets = load_dataset("parquet", data_files={"train": train_files, "validation": validation_files})
datasets = datasets.filter(lambda x: x['text'] is not None and x['reply'] is not None)

Using custom data configuration default-62766e55e41dd593


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /home/jader/.cache/huggingface/datasets/parquet/default-62766e55e41dd593/0.0.0/03dda9603b6ba3760d9d286684a3d7d8ec00448c154f765795485acd3229ecba. Subsequent calls will reuse this data.


  0%|          | 0/166 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [9]:
show_random_elements(datasets["train"])

,id,text,reply
0,/members/hoguga.154856/,Opa.. Segue meus testes: <url> [email protected] </url> /1100 <url> img405.imageshack.us <image> </url> <url> [email protected] </url> Stock <url> img405.imageshack.us <image> </url> <url> img41.imageshack.us <image> </url> <url> img694.imageshack.us <image> </url> Claramente o teste da area 01 força a gpu e o restante testa o processador... <emoji> :D </emoji> abraçoss :yes:,"Do teste 2 ao 4 sua HD 5850 está com gargalo por causa da CPU, podia render mais, porém nada que deva atrapalhar, a não ser pelo fps mínimo que não tem como avaliar por esse bench. Falou, bom divertimento :yes:"
1,/members/rntbarbosa.2499868/,"O ping ficou melhor, ou continua alto? Após o reparo na TAR, a linha voltou a suportar 15mb? O ruido tá alto e o modem entrou em modo adaptativo, 25mb deve alinhar em ~27.xxx Kbps. Já corrigiram o problema? Tô a um mês tentando a substituição meu td5130v1 que uso a uns 4 anos, e agora começou a travar aleatoriamente, a oi mandou 3x um técnico na minha casa, mas ele não tinha modem novo no carro, até fiz uma reclamação na anatel sobre isso, deixei bem claro que o modem precisava ser trocado e que isso era responsabilidade da OI pois, no contrato o modem é cedido em regime de comodato ( <url> www.oi.com.br <link> </url> ), e o que a OI fez?? Abriu outro reparo, e mandou mais um técnico sem modem. Só me resta reclamar no consumidor.gov. Notou se o wifi do pace é mais forte que o do tech? Não achei firmware genérico desse modem, acho que tem como extrair ele do modem por telnet. Se vc poder, extrai ele e posta aqui. Quanto tá seu snr e att? O segundo modem também veio sem firmware personalizado para a OI/fon? No sac não temos como solicitar essa mudança, só o técnico do fixo pode fazer essa troca, se tiver viabilidade, e se ele quiser, pq tem uns que não gostam de fazer esse procedimento, ou seja, depende vontade/pressa do técnico. O snr se manteve o mesmo nos dois, obrigado por postar esse comparativo. Descobriu se é a placa wifi do note que não tá suportando mais do que 2.2mb/s? Atualizou o driver da placa wifi?","Sim, ambos vieram sem FON, pelo que o técnico disse, eles são diferentes pois eram pra ser os modens de teste aqui na região, mas colocaram pra mim pois não tinham outros."
2,/members/rexbauru.222900/,"olhei aqui e o watmam mostra temp máxima 90ºC temp alvo 75ºC , não tentei baixar a voltagem não.","Então realmente o perfil da fan na Sapphire é bem modesto, porque deixar a placa passar 4°C do target é dose... Falouws <emoji> :cool: </emoji>"
3,/members/lu4414.2714338/,"Não há problema nenhum nisso, isso tem a ver com a região. Locais ex-telesp que receberam a rede primeiro (na época do fusionado) tem um over maior e nas regiões novas eles controlam melhor direto na OLT. Isso de expansão tem a ver com concorrência. A meta da Vivo é (e de qualquer outra hehehe) é lucrar e a estrutura de rede deles se alinha com esse objetivo. Não compreendo o porque cabear só 1/3 de um prédio (será que eles não confiavam que a demanda seria maior?), não sei se é falha de planejamento ou se o plano é esse e tudo bem para eles não atender todos.... O que sei que deve crescer com o tempo - e talvez ajude nisso - é a pressão por boas opções de internet na hora de comprar/alugar um espaço, já passou da hora de construtoras e teles se conversarem melhor!","entendi. Porem algo de errado não está certo. hoje minha conexão está muito instável, com dificuldade de carregar algumas paginas que acesso para trabalho e com uma limitação de velocidade bem abaixo da contratada. nos testes que fiz hoje, a velocidade máxima alcançada foi de 220/120mb (meu plano é 300/150) por cabo de rede e com somente meu computador ativo na rede. reiniciei o modem, e não obtive nenhuma melhora de velocidade ou nos carregamentos de pagina. mudei meu cabo de rede para o modem da Claro/NET, e minha navegação ficou normal. o estranho é que mesmo com a velocidade dando 220mb, eu não conseguia abrir algumas paginas de forma 

In [25]:
def tokenize_function(examples):
    result = tokenizer(examples["text"] + examples["reply"],
                       padding="max_length", truncation=True, max_length=64)
    result["labels"] = result["input_ids"].copy()
    return result

In [26]:
tokenized_datasets = datasets.map(
    tokenize_function,
    num_proc=8,
    batched=True,
    batch_size=128,
    remove_columns=["id", "text", "reply"])

# Train transformerslib

In [27]:
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=2000,
    num_train_epochs=20,
    save_strategy="epoch"
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(50000)),
    eval_dataset=tokenized_datasets["validation"].shuffle(seed=42).select(range(1000)),
)

In [29]:
trainer.train()

***** Running training *****
  Num examples = 50000
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 125000


Epoch,Training Loss,Validation Loss
1,0.738600,3.050401
2,0.694000,2.933991
3,0.630000,2.865727
4,0.591600,2.845868
5,0.558700,2.820881
6,0.535200,2.820131
7,0.523100,2.840756
8,0.465100,2.801399
9,0.434700,2.833448
10,0.444600,2.883101


Saving model checkpoint to test-clm/checkpoint-500
Configuration saved in test-clm/checkpoint-500/config.json
Model weights saved in test-clm/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-1000
Configuration saved in test-clm/checkpoint-1000/config.json
Model weights saved in test-clm/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-1500
Configuration saved in test-clm/checkpoint-1500/config.json
Model weights saved in test-clm/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-2000
Configuration saved in test-clm/checkpoint-2000/config.json
Model weights saved in test-clm/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-2500
Configuration saved in test-clm/checkpoint-2500/config.json
Model weights saved in test-clm/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-3000
Configuration saved in test-clm/checkpoint-3000/config.json
M

Model weights saved in test-clm/checkpoint-22500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-23000
Configuration saved in test-clm/checkpoint-23000/config.json
Model weights saved in test-clm/checkpoint-23000/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-23500
Configuration saved in test-clm/checkpoint-23500/config.json
Model weights saved in test-clm/checkpoint-23500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-24000
Configuration saved in test-clm/checkpoint-24000/config.json
Model weights saved in test-clm/checkpoint-24000/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-24500
Configuration saved in test-clm/checkpoint-24500/config.json
Model weights saved in test-clm/checkpoint-24500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-25000
Configuration saved in test-clm/checkpoint-25000/config.json
Model weights saved in test-clm/checkpoint-25000/pytorch_model.bin
***** Running Evaluation ***

Model weights saved in test-clm/checkpoint-44500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-45000
Configuration saved in test-clm/checkpoint-45000/config.json
Model weights saved in test-clm/checkpoint-45000/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-45500
Configuration saved in test-clm/checkpoint-45500/config.json
Model weights saved in test-clm/checkpoint-45500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-46000
Configuration saved in test-clm/checkpoint-46000/config.json
Model weights saved in test-clm/checkpoint-46000/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-46500
Configuration saved in test-clm/checkpoint-46500/config.json
Model weights saved in test-clm/checkpoint-46500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-47000
Configuration saved in test-clm/checkpoint-47000/config.json
Model weights saved in test-clm/checkpoint-47000/pytorch_model.bin
Saving model checkpoint to t

Saving model checkpoint to test-clm/checkpoint-67000
Configuration saved in test-clm/checkpoint-67000/config.json
Model weights saved in test-clm/checkpoint-67000/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-67500
Configuration saved in test-clm/checkpoint-67500/config.json
Model weights saved in test-clm/checkpoint-67500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-68000
Configuration saved in test-clm/checkpoint-68000/config.json
Model weights saved in test-clm/checkpoint-68000/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-68500
Configuration saved in test-clm/checkpoint-68500/config.json
Model weights saved in test-clm/checkpoint-68500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to test-clm/checkpoint-69000
Configuration saved in test-clm/checkpoint-69000/config.json
Model weights saved in test-clm/checkpoint-69000/pytorch_model.bin
Saving model checkpoint t

KeyboardInterrupt: 

In [30]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


Perplexity: 17.70


# Train pytorch

In [ ]:
tokenized_datasets.set_format("torch")

In [ ]:
train_dataloader = DataLoader(tokenized_datasets["train"].shuffle(seed=42).select(range(10000)), batch_size=8)
eval_dataloader = DataLoader(tokenized_datasets["validation"].shuffle(seed=42).select(range(1000)), batch_size=8)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_linear_schedule_with_warmup

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=2000,
    num_training_steps=num_training_steps
)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device);

In [ ]:
progress_bar = tqdm.notebook.tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
metric = load_metric("accuracy")

model.eval()

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    pred = torch.argmax(logits, dim=-1)
    print(tokenizer.decode(pred[0].tolist()))
    print(tokenizer.decode(batch["labels"][0]))
    metric.compute(predictions=pred, references=batch["labels"])

metric.compute()